In [ ]:
# ============================================
# 1) 설치
# ============================================
!pip -q install langchain langchain-community langchain-text-splitters chromadb sentence-transformers pypdf transformers accelerate langchain-huggingface


In [ ]:

# ============================================
# 2) PDF 다운로드 (경제금융용어 700선)
# ============================================
import requests, os, urllib.request

url = "https://github.com/chatgpt-kr/openai-api-tutorial/raw/main/ch07/2020_%EA%B2%BD%EC%A0%9C%EA%B8%88%EC%9C%B5%EC%9A%A9%EC%96%B4%20700%EC%84%A0_%EA%B2%8C%EC%8B%9C.pdf"
pdf_path = "2020_경제금융용어 700선_게시.pdf"

# 다운로드
if not os.path.exists(pdf_path) or os.path.getsize(pdf_path) == 0:
    urllib.request.urlretrieve(url, filename=pdf_path)
print("✅ PDF ready:", pdf_path)

# ============================================
# 3) PDF 로드 & 청크 분할
# ============================================
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader(pdf_path)
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(docs)
print(f"📄 전체 페이지 수: {len(docs)}, 🧩 청크 수: {len(chunks)}")

# ============================================
# 4) 한국어 임베딩 + 벡터스토어
# ============================================
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

embedding = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
chroma_store = Chroma.from_documents(chunks, embedding=embedding)
print("✅ Chroma DB 생성 완료")

# ============================================
# 5) HuggingFacePipeline (google/mt5-small)
# ============================================
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain_huggingface import HuggingFacePipeline

MODEL_NAME = "google/mt5-small"
tok = AutoTokenizer.from_pretrained(MODEL_NAME)
mdl = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

pipe = pipeline(
    "text2text-generation",
    model=mdl,
    tokenizer=tok,
    max_new_tokens=128,
    truncation=True,
    device=-1  # CPU 강제 (GPU 문제 방지)
)
llm = HuggingFacePipeline(pipeline=pipe)

# ============================================
# 6) SelfQueryRetriever 생성
# ============================================
from langchain.chains.query_constructor.schema import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever

metadata_field_info = [
    AttributeInfo(name="page", description="페이지 번호", type="integer"),
]

selfquery_retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=chroma_store,
    document_contents="경제금융 용어 설명 문서",  # ✅ 최신 버전에서는 document_contents 사용
    metadata_field_info=metadata_field_info,
    verbose=True,
)

# ============================================
# 7) 질의 테스트
# ============================================
def ask(q, k=3):
    docs = selfquery_retriever.get_relevant_documents(q)
    print(f"\n🔎 Q: {q}\n--- 결과 ---")
    for i, d in enumerate(docs[:k], 1):
        page = d.metadata.get("page", "?")
        txt = d.page_content[:120].replace("\n"," ")
        print(f"{i:>2}. p.{page} | {txt}...")

# 테스트 질문
ask("20페이지 이후에 나오는 이자 관련 용어를 찾아줘")
ask("인플레이션과 금리 정책의 관계를 보여줘")
